## Import Library

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

## Load Data

In [4]:
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
data = pd.read_csv("iris.csv", skiprows=1, header=None, names=col_names)
data.head(10)

,sepal_length,sepal_width,petal_length,petal_width,type
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa
10,4.9,3.1,1.5,0.1,Iris-setosa


## Node Class

In [5]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        # Constructor Kelas Node/Cabang 
        
        # Percabagan
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # Pembuatan Daun
        self.value = value

## Tree Class

In [6]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        # Constructor Kelas Model
        
        # Inisialisasi Akar Dari Model 
        self.root = None
        
        # Kondisi Pemberhentian Pembentukan Model
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        # Fungsi Rekursif Untuk Membangun Model Menggunakan Regresi
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # Melakukan Percabangan Sampai Kondisi Berhenti Terpenuhi
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # Menentukan Percabangan Terbaik
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # Melakukan Pengecekan Apakah Nilai Information Gain Bernilai Positive
            if best_split["info_gain"]>0:
                # Percabangan ke Kiri
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # Percabangan ke Kanan
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # Pengembalian Nilai Cabang (Fitur & Information Gain)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # Menghitung Banyaknya Daun
        leaf_value = self.calculate_leaf_value(Y)
        # return Jumlah Daun
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        # Fungsi Untuk Menentukan Percabangan Terbaik
        
        # Dictionary Untuk Menyimpan Nilai Information Gain Awal
        best_split = {}
        max_info_gain = -float("inf")
        
        # Looping Untuk Menghitung Setiap Value Dalam Suatu Feature
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # Looping Untuk Setiap Fitur yang Tersedia Dalam Dataset
            for threshold in possible_thresholds:
                # Pengecekan Pembagian Sample Data Kanan dan Kiri
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # Pengecekan Apakah Sample Data Pada Setiap Cabang Kosong atau Tidak
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # Menghitung Information Gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # Mengupdate Nilai Best Split Jika Diperlukan
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return Nilai Best Split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        # Fungsi Untuk Split Sample Data
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        # Fungsi Untuk Menghitung Information Gain Menggunakan Gini Index
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        # Fungsi Untuk Menghitung Entropy Pada Features
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        # Fungsi Untuk Menghitung Gini Index
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        # Fungsi Untuk Menghitung Jumlah Cabang/Daun
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        # Fungsi Untuk Mencetak Model
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        # Fungsi Untuk Melakukan Training/Pembentukan Model
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        # Fungsi Untuk Melakukan Prediksi
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        # Fungsi Untuk Melakukan Prediksi Dengan 1 Data
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)



## Split Data

In [24]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=41)

## Fit Data

In [25]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_2 <= 1.9 ? 0.33741385372714494
 left:Iris-setosa
 right:X_3 <= 1.5 ? 0.427106638180289
  left:X_2 <= 4.9 ? 0.05124653739612173
    left:Iris-versicolor
    right:Iris-virginica
  right:X_2 <= 5.0 ? 0.019631171921475288
    left:X_1 <= 2.8 ? 0.20833333333333334
        left:Iris-virginica
        right:Iris-versicolor
    right:Iris-virginica


## Testing Model

In [26]:
Y_pred = classifier.predict(X_test)
print("Kelas Sesungguhnya : \n", Y_test)
print("Hasil Prediksi : \n", Y_pred)
print("Akurasi : ", accuracy_score(Y_test, Y_pred))

Kelas Sesungguhnya : 
 [['Iris-virginica']
 ['Iris-virginica']
 ['Iris-virginica']
 ['Iris-versicolor']
 ['Iris-virginica']
 ['Iris-versicolor']
 ['Iris-virginica']
 ['Iris-versicolor']
 ['Iris-virginica']
 ['Iris-virginica']
 ['Iris-virginica']
 ['Iris-setosa']
 ['Iris-setosa']
 ['Iris-versicolor']
 ['Iris-setosa']
 ['Iris-virginica']
 ['Iris-setosa']
 ['Iris-versicolor']
 ['Iris-setosa']
 ['Iris-setosa']
 ['Iris-versicolor']
 ['Iris-virginica']
 ['Iris-setosa']
 ['Iris-setosa']
 ['Iris-versicolor']
 ['Iris-versicolor']
 ['Iris-versicolor']
 ['Iris-versicolor']
 ['Iris-setosa']
 ['Iris-versicolor']]
Hasil Prediksi : 
 ['Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-

## Confussion Matrix

In [27]:
pd.DataFrame(
    confusion_matrix(Y_test, Y_pred),
    index=['True : Iris-setosa', 'True : Iris-versicolor', 'True : Iris-virginica'],
    columns=['Pred : Iris-setosa', 'Pred : Iris-versicolor', 'Pred : Iris-virginica'],
)

,Pred : Iris-setosa,Pred : Iris-versicolor,Pred : Iris-virginica
True : Iris-setosa,9,0,0
True : Iris-versicolor,0,10,1
True : Iris-virginica,0,1,9
